#  Retrieval-Augmented Generation (RAGS)

In [ ]:
![title]("img/picture.png")

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage
import bs4

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install faiss-cpu
!pip install bs4
!pip install langchain-groq

In [2]:
# Document location
urls = [
    "https://github.com/blueraincloud/blueraincloud.github.io/blob/main/misc/rag-text.txt"
]

#  User Agent
class CustomWebBaseLoader(WebBaseLoader):
    def __init__(self, url):
        super().__init__(url, requests_kwargs={"headers": {"User-Agent": "Mozilla/5.0"}})

# Load document
docs = [CustomWebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [60]:
# Text_splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)

In [62]:
vectorstore = FAISS.from_documents(doc_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever = vectorstore.as_retriever(k=4)

In [63]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [68]:
# Initialize Llama 3.1
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

In [72]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an assistant for question-answering tasks.
            Use short sentences and keep the answer to a max of three sentences.
            Question: {question}
            Answer:""",
        ),
        ("human", "{question}"),
    ]
)

chain = prompt | llm
ai_msg = chain.invoke(
    {
        "question": "What is prompt-engineering?",
    }
)
ai_msg.content

'Prompt-engineering is the process of designing and refining input prompts to elicit accurate and relevant responses from language models, such as chatbots or virtual assistants. It involves understanding how to craft effective questions or statements that can be understood by AI systems. This technique helps improve the quality and relevance of generated answers.'

In [73]:
ai_msg = chain.invoke(
    {
        "question": "What is clorkimn?",
    }
)
ai_msg.content

'I couldn\'t find any information on "clorkimn". It\'s possible it\'s a misspelling or not a widely known term.'

In [74]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the documents in question and fall back on the main corpus of knowledge if the documents are insufficient.
    Do not mention when you fall back or if it comes from the documents or main corpus of knowledge.
    Use short sentences and keep the answer to a max of three sentences.
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [75]:
# Create a chain combining the prompt template and LLM
rag_chain = prompt | llm | StrOutputParser()

In [76]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Retrieve relevant documents
        documents = self.retriever.invoke(question)
        # Extract content from retrieved documents
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        #print(doc_texts)
        # Get the answer from the language model
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [77]:
# Initialize the RAG application
rag_application = RAGApplication(retriever, rag_chain)
# Example usage
question = "What is Prompt engineering"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What is Prompt engineering
Answer: Prompt engineering is a way of life that involves cultivating awareness, fostering creativity, and connecting with others through shared experiences. It's about harnessing the power of your thoughts and feelings to connect with the universe by crafting the right questions to unlock the treasures of your mind and surroundings. This practice helps you engage with the world around you and discover new insights about yourself and the world.


In [79]:
question = "What is clorkimn?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What is clorkimn?
Answer: Clorkimn refers to a word that means green blue in fantasy worlds. It comes from the English words clorkimness and is an uncommon term that was lost in translation but remains important. Clorkimn is associated with a specific color palette found in imaginative realms.


## DORA

In [163]:
# importing required classes
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document  # Import the Document class
doc_names = [
    "DORA/a.pdf",
    "DORA/b.pdf",
    "DORA/c.pdf",
    "DORA/d.pdf",
    "DORA/e.pdf",
    "DORA/f.pdf",
    "DORA/g.pdf",
    "DORA/h.pdf",
    "DORA/i.pdf",
    "DORA/j.pdf"
]
corpus = []
for adoc in doc_names:
    # creating a pdf reader object
    reader = PdfReader(adoc)
    
    # printing number of pages in pdf file
    print(len(reader.pages))
    
    # creating a page object
    page = reader.pages[1]
    
    # extracting text from page
    #print(page.extract_text())

    docs_list = [Document(page_content=doc.extract_text()) for doc in reader.pages]
    corpus = corpus + docs_list


77
57
113
182
184
128
26
48
36
21


In [164]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(corpus)

In [165]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

vectorstore = FAISS.from_documents(doc_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever = vectorstore.as_retriever(k=4)

In [166]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant for question-answering tasks. \
            Use the following documents to answer the question.\
            If you don't know the answer, just say that you don't know.\
            In general you are dealing with regulatory documents and often it is verbose. Giving a summary would be helpful.\
            Each document typicall has a background and rational and a technical standards section.\
            WHen giving answer back focus on the content in the technical standards section.\
            THe background and rational and other sections are not as important:\
            Question: {question}\
            Answer:",
        ),
        ("human", "{question}"),
    ]
)

chain = prompt | llm
ai_msg = chain.invoke(
    {
        "question": "What is DORA act about?",
    }
)
ai_msg.content

'I don\'t have any documents related to a "DORA act". Could you please provide more context or information about what DORA act refers to? I\'ll do my best to find relevant documents and answer your question. \n\nHowever, after some research, I found that the Digital Operational Resilience Act (DORA) is a proposed EU regulation aimed at strengthening the operational resilience of financial institutions. If this is the correct context, please let me know and I can try to provide more information based on available documents.\n\nPlease note that my previous response was incorrect, and I\'m trying to correct it now.'

In [167]:
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. \
            Use the following documents to answer the question.\
            If you don't know the answer, just say that you don't know.\
            In general you are dealing with regulatory documents and often it is verbose. Giving a summary would be helpful.\
             Each document typicall has a section where responds give their feedback and a section where the actual draft of standards are given.\
            When summarizing this information only consider the actual standards that are set and ignore the other sections:\
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [168]:
# Create a chain combining the prompt template and LLM
rag_chain = prompt | llm | StrOutputParser()

rag_application = RAGApplication(retriever, rag_chain)
# Example usage
question = "What is DORA act about?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What is DORA act about?
Answer: Based on the provided documents, I can summarize that DORA is about regulations related to the oversight of financial entities and ICT third-party service providers.

The actual standards set by DORA include:

* Tests organized at the level of a financial entity by the TLPT authority of its home Member State (point 75)
* Requirements for competent authorities in relation to the joint examination team (point c)

These regulations are outlined in two separate Regulatory Technical Standards (RTS) under DORA.

As for what DORA is about, I can provide a brief summary:

DORA appears to be an act that regulates the oversight of financial entities and ICT third-party service providers. It sets standards for tests and examinations to be conducted by competent authorities, with a focus on ensuring the stability and security of the financial system.


In [169]:
# Example usage
question = "Can you give me a one page summary of the documents"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Can you give me a one page summary of the documents
Answer: Here is a one-page summary of the documents:

**Summary**

The European Securities and Markets Authority (ESAs) has made some changes to provide more clarity in the draft regulatory technical standards (RTS). The main points are:

* **Electronic format**: The report must be in a searchable electronic format, but no specific document type is mandated.
* **Examples or best practices**: These are out of scope for the RTS.
* **Purpose of "start" and "end" dates**: The text is considered sufficiently clear.

**Key Changes**

The ESAs have introduced some key changes to provide more clarity:

* Paragraph 1 only requires a searchable electronic format, without mandating a specific file type.
* Paragraph 2 elaborates on the content expected from the report, covering minimum elements that must be included. Entities may include additional information if deemed useful.

**Contractual Structure and Documentation**

The preferred

In [170]:
# Example usage
question = "What is the register of information"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What is the register of information
Answer: Based on the documents provided, here is a summary of what I found regarding the "register of information":

**Summary:** The register of information is composed of 15 templates that are linked together using relational keys. The templates cover three purposes: (i) ICT risk management; (ii) reporting and disclosure; and (iii) supervision.

**Key Components:**

1. **Templates**: There are 15 templates in total, which are linked to each other using relational keys.
2. **Relational Keys**: Some of the relational keys used include:
	* Contractual arrangement reference number
	* LEI (Legal Entity Identifier) of the entity making use of ICT services
	* ICT third-party service provider identifier
	* Function identifier
	* Type of ICT services (provided in Annex III)
3. **Purpose**: The register of information serves three purposes:
	* ICT risk management
	* Reporting and disclosure
	* Supervision

**Standards:**

1. Financial entities must

In [171]:
# Example usage
question = "What can you tell me about the standards on classifying ICT related incidients"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What can you tell me about the standards on classifying ICT related incidients
Answer: Based on the provided document, here's a summary of the standards related to classifying ICT-related incidents:

**Classification Criteria**

The draft RTS (Regulatory Technical Standard) specifies criteria for classifying ICT-related incidents under DORA (Digital Operational Resilience Act). The classification is based on the following criteria:

* Major ICT-related incidents: These are incidents that have a significant impact on costs and losses, and financial entities must report them in their reference year or previous years if they had an impact on costs and losses.
* Significant cyber threats: These are threats that have a potential to cause major ICT-related incidents.

**Materiality Thresholds**

The draft RTS also sets out materiality thresholds for major incidents and significant cyber threats. Financial entities must assess gross costs and losses using the same approach as the re

In [172]:
# Example usage
question = "What can you tell me about the RTS on ICT services supporting critical or important functions"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What can you tell me about the RTS on ICT services supporting critical or important functions
Answer: Based on the provided documents (134, 156, 71, and 147), I can summarize the information related to the Regulatory Technical Standard (RTS) on Information and Communication Technology (ICT) services supporting critical or important functions.

**Summary:**

The RTS aims to specify criteria for classifying ICT-related incidents, materiality thresholds for major incidents, and significant cyber threats under the Digital Operational Resilience Act (DORA). The key points related to ICT services supporting critical or important functions are:

* Some respondents suggested limiting the provisions to ICT assets that serve critical or important functions.
* There was a suggestion to add further requirements, including the expected End-of-Life of ICT assets, especially for Legacy Systems. However, the European Supervisory Authorities (ESAs) decided not to include these additional requ

In [173]:
# Example usage
question = "What can you tell me about the risk management framework and simplified risk management framework. What are the main differences? Ignore the correspondance and give me the standards"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What can you tell me about the risk management framework and simplified risk management framework. What are the main differences? Ignore the correspondance and give me the standards
Answer: Based on the provided documents, here's a summary of the risk management framework and simplified risk management framework:

**Risk Management Framework:**

* The draft RTS (Regulatory Technical Standard) should take a principle-based and objective-focused approach.
* It should provide high-level principles and objectives for financial entities to develop and customize their risk management framework.
* The approach aims to ensure consistency and uniformity in risk management practices across the industry, facilitating easier supervision and regulatory oversight.

**Simplified Risk Management Framework:**

* The draft RTS based on Article 16 of DORA (Digital Operational Resilience Act) includes a simplified ICT (Information and Communication Technology) risk management framework.
* This f

In [174]:
# Example usage
question = "TLPT is threat led penetration testing. What can you tell me about the TLPT. What are the main differences?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: TLPT is threat led penetration testing. What can you tell me about the TLPT. What are the main differences?
Answer: Based on the provided documents, here's a summary of TLPT (Threat Led Penetration Testing) and its main differences:

**TLPT Overview:**

TLPT is a threat-led penetration testing method that involves simulating cyber attacks on live production systems to identify vulnerabilities. It is designed to be more advanced than less sophisticated testing methods covered by Article 24 of the Digital Operational Resilience Act (DORA).

**Key Requirements and Differences:**

1. **Live Production Systems:** TLPT must be carried out on live production systems, as per Articles 3(17) and 26(2) of DORA.
2. **Advanced Testing Methodology:** TLPT is a more advanced testing method than less sophisticated testing methods covered by Article 24 of DORA.
3. **Risk Management:** The control team must conduct an assessment of risks associated with the testing, including potential impacts

In [175]:
# Example usage
question = "How does DORA treat estimation of aggregated annual costs and losses caused by major ICT-related incidents?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)




Question: How does DORA treat estimation of aggregated annual costs and losses caused by major ICT-related incidents?
Answer: Based on the provided documents, here's a summary of how DORA treats estimation of aggregated annual costs and losses caused by major ICT-related incidents:

According to Article 11(10) of Regulation 2022/2554 (DORA), financial entities are mandated to report an estimation of aggregated annual costs and losses caused by major ICT-related incidents to the competent authorities upon request.

The guidelines aim to harmonize the estimation of these costs and losses across sectors, as reported data may be based on different methodologies and assumptions, leading to a lack of comparability. Financial entities should aggregate gross costs and losses, as well as financial recoveries, across major ICT-related incidents.

To estimate these costs and losses, financial entities should refer to their financial statements, such as the profit and loss account, or supervisory 

In [177]:
# Example usage
question = "What does DORA say about third-party risk management and contract management?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)




Question: What does DORA say about third-party risk management and contract management?
Answer: Based on the provided documents, here's a summary of what DORA says about third-party risk management and contract management:

**Third-Party Risk Management:**

* Financial entities must comply with Chapter V "Managing of ICT third-party risk" of DORA.
* Key principles for sound management of ICT third-party risk include:
	+ Regular review of policy (at least once a year).
	+ Conducting risk assessments and maintaining internal responsibilities, skills, experience, and knowledge to ensure effective monitoring and oversight of contractual arrangements.

**Contract Management:**

* The RTS specifies detailed content on contractual arrangements regarding the use of ICT services supporting critical or important functions provided by ICT third-party service providers.
* Governance requirements aim to ensure that contractual arrangements do not impede financial entities from fulfilling DORA requi

## Next steps

* Evaluation metrics
* Fine-tuning comparison

## References

 - https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Getting_to_know_Llama.ipynb
 - https://www.datacamp.com/tutorial/llama-3-1-rag